In [82]:
!pip install mlxtend

In [83]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option('display.max_columns', None)

In [84]:
#Filtered MLB Hitters Data for 2022 - 2024 Seasons
Csv = pd.read_csv('BatterInformation.csv')
filteredCSV = Csv[(Csv['year'] >= 2022) & (Csv['year'] < 2025)]




In [85]:
#Loading Pitcher Data From 2022 - 2024 Seasons
Pitchers = pd.read_csv('Pitchers_With_Clusters.csv')
filteredPitchers = Pitchers[(Pitchers['year'] >= 2022) & (Pitchers['year'] < 2025)]



In [86]:
display(filteredPitchers)

,player_id,year,"last_name, first_name",kmeans_quality_cluster,quality_archetype,impact_cluster,impact_archetype
4,425794,2022,"Wainwright, Adam",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
5,425794,2023,"Wainwright, Adam",5,Sinkers Get Hit (Bad SI),1,Sinker Liability Pitchers (Sinker Gets Crushed)
9,425844,2022,"Greinke, Zack",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)
10,425844,2023,"Greinke, Zack",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)
17,434378,2022,"Verlander, Justin",7,Slider Aces (Elite SL),5,Four-Seam Aces (Elite FF Impact)
...,...,...,...,...,...,...,...
2208,695243,2024,"Miller, Mason",2,Four-Seam Aces (Elite FF Quality),5,Four-Seam Aces (Elite FF Impact)
2212,700249,2024,"Povich, Cade",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
2215,701581,2024,"Festa, David",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)
2220,806185,2024,"Birdsong, Hayden",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)


In [87]:
display(filteredCSV)

,"last_name, first_name",player_id,year,Cluster_Group,Hitter_Types,Cluster_Group.1,Ball Preference,Cluster_Group.2,Swinging Tendencies
982,"Cruz, Nelson",443558,2022,2,Balanced Hitters,2,Aggressive Fastball,1,Balanced
983,"Blackmon, Charlie",453568,2022,0,Hitters With Good Contact,2,Aggressive Fastball,0,Aggressive
984,"McCutchen, Andrew",457705,2022,2,Balanced Hitters,2,Aggressive Fastball,1,Balanced
985,"Turner, Justin",457759,2022,2,Balanced Hitters,2,Aggressive Fastball,2,Selective
986,"Andrus, Elvis",462101,2022,0,Hitters With Good Contact,0,Selective / Moderate,2,Selective
...,...,...,...,...,...,...,...,...,...
1370,"Chourio, Jackson",694192,2024,2,Balanced Hitters,0,Selective / Moderate,0,Aggressive
1371,"Schanuel, Nolan",694384,2024,0,Hitters With Good Contact,0,Selective / Moderate,2,Selective
1372,"Langford, Wyatt",694671,2024,2,Balanced Hitters,0,Selective / Moderate,1,Balanced
1373,"Young, Jacob",696285,2024,0,Hitters With Good Contact,0,Selective / Moderate,2,Selective


In [88]:
#2022 Data
Events = pd.read_csv('statcast_pitch_by_pitch_2022.csv')
CondensedEvent = Events[['game_date','batter','pitcher','events','pitch_name','year']].copy()
CondensedEvent_no_out = CondensedEvent[~CondensedEvent['events'].str.contains('out|double_play|triple_play', case=False, na=False)]



In [89]:
merged = CondensedEvent_no_out.merge(filteredCSV, left_on= ["batter",'year'], right_on=["player_id",'year'], how="left")
df_clean = merged.dropna(subset=['pitch_name', 'events', 'Hitter_Types'])
complete = df_clean.merge(filteredPitchers,left_on = ['pitcher','year'], right_on = ['player_id','year'])


In [90]:
display(complete)

,game_date,batter,pitcher,events,pitch_name,year,"last_name, first_name_x",player_id_x,Cluster_Group,Hitter_Types,Cluster_Group.1,Ball Preference,Cluster_Group.2,Swinging Tendencies,player_id_y,"last_name, first_name_y",kmeans_quality_cluster,quality_archetype,impact_cluster,impact_archetype
0,2022-11-05,608324,502085,single,Knuckle Curve,2022,"Bregman, Alex",608324.0,2.0,Balanced Hitters,0.0,Selective / Moderate,2.0,Selective,502085,"Robertson, David",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
1,2022-11-05,514888,621107,single,Curveball,2022,"Altuve, Jose",514888.0,0.0,Hitters With Good Contact,0.0,Selective / Moderate,2.0,Selective,621107,"Eflin, Zach",0,Fastball Generalists (Good FF + Good SI),2,Balanced Positive Arsenal (Mild Impact Gains)
2,2022-11-05,608324,621237,walk,Cutter,2022,"Bregman, Alex",608324.0,2.0,Balanced Hitters,0.0,Selective / Moderate,2.0,Selective,621237,"Alvarado, José",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
3,2022-11-05,670541,621237,home_run,Sinker,2022,"Alvarez, Yordan",670541.0,1.0,Power Hitters,0.0,Selective / Moderate,1.0,Balanced,621237,"Alvarado, José",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
4,2022-11-05,665161,554430,single,Sinker,2022,"Peña, Jeremy",665161.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,554430,"Wheeler, Zack",9,Cutter Aces (Elite FC),5,Four-Seam Aces (Elite FF Impact)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20282,2022-03-19,665489,605400,home_run,Changeup,2022,"Guerrero Jr., Vladimir",665489.0,1.0,Power Hitters,0.0,Selective / Moderate,0.0,Aggressive,605400,"Nola, Aaron",0,Fastball Generalists (Good FF + Good SI),5,Four-Seam Aces (Elite FF Impact)
20283,2022-03-19,668804,656638,single,Knuckle Curve,2022,"Reynolds, Bryan",668804.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,656638,"Lange, Alex",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
20284,2022-03-18,650402,608717,single,4-Seam Fastball,2022,"Torres, Gleyber",650402.0,2.0,Balanced Hitters,0.0,Selective / Moderate,1.0,Balanced,608717,"Stratton, Chris",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
20285,2022-03-18,650402,640444,walk,Changeup,2022,"Torres, Gleyber",650402.0,2.0,Balanced Hitters,0.0,Selective / Moderate,1.0,Balanced,640444,"Crowe, Wil",6,Weak Breaking Ball Group (Bad SL),2,Balanced Positive Arsenal (Mild Impact Gains)


In [91]:
#2023
Events2023 = pd.read_csv('statcast_pitch_by_pitch_2023.csv')
CondensedEvent2023 = Events2023[['game_date','batter','pitcher','events','pitch_name','year']].copy()
CondensedEvent_no_out2023 = CondensedEvent2023[~CondensedEvent2023['events'].str.contains('out|double_play|triple_play', case=False, na=False)]

display(CondensedEvent_no_out2023)

,game_date,batter,pitcher,events,pitch_name,year
1,2023-11-01,606466,622250,NaN,4-Seam Fastball,2023
2,2023-11-01,606466,622250,NaN,Curveball,2023
3,2023-11-01,606466,622250,NaN,4-Seam Fastball,2023
4,2023-11-01,606466,622250,NaN,4-Seam Fastball,2023
6,2023-11-01,682998,622250,NaN,Curveball,2023
...,...,...,...,...,...,...
774031,2023-03-15,670231,641540,NaN,NaN,2023
774032,2023-03-15,670231,641540,NaN,NaN,2023
774035,2023-03-15,668472,641540,NaN,NaN,2023
774036,2023-03-15,668472,641540,NaN,NaN,2023


In [92]:
merged2 = CondensedEvent_no_out2023.merge(filteredCSV, left_on= ["batter",'year'], right_on=["player_id",'year'], how="left")
df_clean2 = merged2.dropna(subset=['pitch_name', 'events', 'Hitter_Types'])
complete2 = df_clean2.merge(filteredPitchers,left_on = ['pitcher','year'], right_on = ['player_id','year'])

In [93]:
display(complete2)

,game_date,batter,pitcher,events,pitch_name,year,"last_name, first_name_x",player_id_x,Cluster_Group,Hitter_Types,Cluster_Group.1,Ball Preference,Cluster_Group.2,Swinging Tendencies,player_id_y,"last_name, first_name_y",kmeans_quality_cluster,quality_archetype,impact_cluster,impact_archetype
0,2023-11-01,608369,623149,single,Sweeper,2023,"Seager, Corey",608369.0,1.0,Power Hitters,0.0,Selective / Moderate,0.0,Aggressive,623149,"Sewald, Paul",2,Four-Seam Aces (Elite FF Quality),5,Four-Seam Aces (Elite FF Impact)
1,2023-11-01,543760,623149,home_run,4-Seam Fastball,2023,"Semien, Marcus",543760.0,2.0,Balanced Hitters,0.0,Selective / Moderate,2.0,Selective,623149,"Sewald, Paul",2,Four-Seam Aces (Elite FF Quality),5,Four-Seam Aces (Elite FF Impact)
2,2023-11-01,663993,623149,single,4-Seam Fastball,2023,"Lowe, Nathaniel",663993.0,2.0,Balanced Hitters,2.0,Aggressive Fastball,1.0,Balanced,623149,"Sewald, Paul",2,Four-Seam Aces (Elite FF Quality),5,Four-Seam Aces (Elite FF Impact)
3,2023-11-01,673962,623149,single,4-Seam Fastball,2023,"Jung, Josh",673962.0,1.0,Power Hitters,0.0,Selective / Moderate,0.0,Aggressive,623149,"Sewald, Paul",2,Four-Seam Aces (Elite FF Quality),5,Four-Seam Aces (Elite FF Impact)
4,2023-11-01,608369,656464,walk,Slider,2023,"Seager, Corey",608369.0,1.0,Power Hitters,0.0,Selective / Moderate,0.0,Aggressive,656464,"Ginkel, Kevin",7,Slider Aces (Elite SL),4,Slider Aces (Elite Slider Impact)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20737,2023-03-15,683011,605400,walk,4-Seam Fastball,2023,"Volpe, Anthony",683011.0,2.0,Balanced Hitters,0.0,Selective / Moderate,1.0,Balanced,605400,"Nola, Aaron",2,Four-Seam Aces (Elite FF Quality),2,Balanced Positive Arsenal (Mild Impact Gains)
20738,2023-03-15,592206,622663,walk,4-Seam Fastball,2023,"Castellanos, Nick",592206.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,622663,"Severino, Luis",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)
20739,2023-03-15,592206,622663,single,Changeup,2023,"Castellanos, Nick",592206.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,622663,"Severino, Luis",3,Weak Fastball Group (Bad FF),0,FF Liability Pitchers (Bad Fastball Impact)
20740,2023-03-15,669261,592332,walk,Split-Finger,2023,"Suwinski, Jack",669261.0,1.0,Power Hitters,0.0,Selective / Moderate,1.0,Balanced,592332,"Gausman, Kevin",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)


In [94]:
#2024
Events2024 = pd.read_csv('statcast_pitch_by_pitch_2024.csv')
CondensedEvent2024 = Events2024[['game_date','batter','pitcher','events','pitch_name','year']].copy()
CondensedEvent_no_out2024 = CondensedEvent2024[~CondensedEvent2024['events'].str.contains('out|double_play|triple_play', case=False, na=False)]
display(CondensedEvent_no_out2024)

,game_date,batter,pitcher,events,pitch_name,year
1,2024-10-30,657077,621111,NaN,Knuckle Curve,2024
2,2024-10-30,657077,621111,NaN,Cutter,2024
3,2024-10-30,657077,621111,NaN,Knuckle Curve,2024
5,2024-10-30,669224,621111,NaN,4-Seam Fastball,2024
6,2024-10-30,669224,621111,NaN,Knuckle Curve,2024
...,...,...,...,...,...,...
760238,2024-03-15,521692,595881,NaN,NaN,2024
760239,2024-03-15,521692,595881,NaN,NaN,2024
760243,2024-03-15,666310,663903,NaN,NaN,2024
760244,2024-03-15,666310,663903,NaN,NaN,2024


In [95]:
merged3 = CondensedEvent_no_out2024.merge(filteredCSV, left_on= ["batter",'year'], right_on=["player_id",'year'], how="left")
df_clean3 = merged3.dropna(subset=['pitch_name', 'events', 'Hitter_Types'])
complete3 = df_clean3.merge(filteredPitchers,left_on = ['pitcher','year'], right_on = ['player_id','year'])

In [96]:
display(complete3)

,game_date,batter,pitcher,events,pitch_name,year,"last_name, first_name_x",player_id_x,Cluster_Group,Hitter_Types,Cluster_Group.1,Ball Preference,Cluster_Group.2,Swinging Tendencies,player_id_y,"last_name, first_name_y",kmeans_quality_cluster,quality_archetype,impact_cluster,impact_archetype
0,2024-10-30,606192,596133,single,4-Seam Fastball,2024,"Hernández, Teoscar",606192.0,1.0,Power Hitters,0.0,Selective / Moderate,1.0,Balanced,596133,"Weaver, Luke",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
1,2024-10-30,605141,596133,sac_fly,4-Seam Fastball,2024,"Betts, Mookie",605141.0,2.0,Balanced Hitters,0.0,Selective / Moderate,2.0,Selective,596133,"Weaver, Luke",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
2,2024-10-30,660271,596133,catcher_interf,Changeup,2024,"Ohtani, Shohei",660271.0,1.0,Power Hitters,0.0,Selective / Moderate,1.0,Balanced,596133,"Weaver, Luke",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
3,2024-10-30,669257,592454,walk,Changeup,2024,"Smith, Will",669257.0,2.0,Balanced Hitters,2.0,Aggressive Fastball,2.0,Selective,592454,"Kahnle, Tommy",8,Changeup Specialists (Elite CH),2,Balanced Positive Arsenal (Mild Impact Gains)
4,2024-10-30,606192,605280,walk,Sinker,2024,"Hernández, Teoscar",606192.0,1.0,Power Hitters,0.0,Selective / Moderate,1.0,Balanced,605280,"Holmes, Clay",1,League-Average Pitch Quality,2,Balanced Positive Arsenal (Mild Impact Gains)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19270,2024-03-15,605137,571927,single,Sinker,2024,"Bell, Josh",605137.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,571927,"Matz, Steven",0,Fastball Generalists (Good FF + Good SI),3,Sinker Aces (Elite Sinker Impact)
19271,2024-03-15,650333,571927,double,Changeup,2024,"Arraez, Luis",650333.0,0.0,Hitters With Good Contact,2.0,Aggressive Fastball,2.0,Selective,571927,"Matz, Steven",0,Fastball Generalists (Good FF + Good SI),3,Sinker Aces (Elite Sinker Impact)
19272,2024-03-15,650559,571927,walk,Slider,2024,"De La Cruz, Bryan",650559.0,2.0,Balanced Hitters,0.0,Selective / Moderate,0.0,Aggressive,571927,"Matz, Steven",0,Fastball Generalists (Good FF + Good SI),3,Sinker Aces (Elite Sinker Impact)
19273,2024-03-15,665862,571927,single,Sinker,2024,"Chisholm Jr., Jazz",665862.0,2.0,Balanced Hitters,0.0,Selective / Moderate,1.0,Balanced,571927,"Matz, Steven",0,Fastball Generalists (Good FF + Good SI),3,Sinker Aces (Elite Sinker Impact)


In [97]:
#Finding the Rules
#Outcomes 2022
transactions22 = complete[['events','Swinging Tendencies','quality_archetype']].copy()


In [98]:

for col in transactions22.columns:
    transactions22[col] = transactions22[col].astype(str)
    transactions22[col] = col + "_" + transactions22[col]

In [99]:
transactionLists = transactions22.values.tolist()

In [100]:
#One Hot Encoding 
EncodeData= TransactionEncoder()
listofItems = EncodeData.fit(transactionLists).transform(transactionLists)
EncodedInformation = pd.DataFrame(listofItems, columns=EncodeData.columns_)

In [101]:
#Apirori ItemSet
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)


In [102]:
#Rules (Swinging Techinque and quality_archetype)	
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)

In [103]:
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
firstCleaned = associationRules.loc[eventsFilter]
firstCleaned.sort_values(by='lift', ascending=False, inplace=True)
firstCleaned.sort_values(by='confidence', ascending=False, inplace=True)
display(firstCleaned)
firstCleaned.to_csv("SwingQuality2022.csv", index=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
226,(quality_archetype_Fastball Generalists (Good ...,(events_single),0.021492,0.432297,0.011387,0.529817,1.225586,1.0,0.002096,1.207408,0.188106,0.025738,0.171780,0.278078
88,(quality_archetype_Changeup Specialists (Elite...,(events_single),0.025879,0.432297,0.012964,0.500952,1.158817,1.0,0.001777,1.137574,0.140692,0.029119,0.120936,0.265470
92,(quality_archetype_Fastball Generalists (Good ...,(events_single),0.053729,0.432297,0.026618,0.495413,1.146002,1.0,0.003391,1.125085,0.134635,0.057940,0.111178,0.278493
142,(quality_archetype_Weak Breaking Ball Group (B...,(events_single),0.028688,0.432297,0.013457,0.469072,1.085070,1.0,0.001055,1.069267,0.080716,0.030069,0.064780,0.250101
90,(quality_archetype_Cutter Aces (Elite FC)),(events_single),0.025829,0.432297,0.012077,0.467557,1.081566,1.0,0.000911,1.066224,0.077414,0.027075,0.062111,0.247747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,(quality_archetype_League-Average Pitch Quality),(events_home_run),0.383053,0.097797,0.035294,0.092137,0.942133,1.0,-0.002168,0.993766,-0.090542,0.079212,-0.006273,0.226512
53,(Swinging Tendencies_Selective),(events_home_run),0.375856,0.097797,0.028097,0.074754,0.764383,1.0,-0.008661,0.975096,-0.330597,0.063060,-0.025540,0.181026
50,(Swinging Tendencies_Selective),(events_hit_by_pitch),0.375856,0.032435,0.013605,0.036197,1.115992,1.0,0.001414,1.003903,0.166526,0.034470,0.003888,0.227825
80,(quality_archetype_League-Average Pitch Quality),(events_hit_by_pitch),0.383053,0.032435,0.012816,0.033458,1.031545,1.0,0.000392,1.001059,0.049568,0.031828,0.001057,0.214297


In [104]:
#Second One (Hitter_Types + Impact Archetype)
transactions22Part2 = complete[['events','Hitter_Types','impact_archetype']].copy()
display(transactions22Part2)
for col in transactions22Part2.columns:
    transactions22Part2[col] = transactions22Part2[col].astype(str)
    transactions22Part2[col] = col + "_" + transactions22Part2[col]
transactionListsPart2 = transactions22Part2.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart2 = EncodeData.fit(transactionListsPart2).transform(transactionListsPart2)
EncodedInformation = pd.DataFrame(listofItemsPart2, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)


,events,Hitter_Types,impact_archetype
0,single,Balanced Hitters,Balanced Positive Arsenal (Mild Impact Gains)
1,single,Hitters With Good Contact,Balanced Positive Arsenal (Mild Impact Gains)
2,walk,Balanced Hitters,Balanced Positive Arsenal (Mild Impact Gains)
3,home_run,Power Hitters,Balanced Positive Arsenal (Mild Impact Gains)
4,single,Balanced Hitters,Four-Seam Aces (Elite FF Impact)
...,...,...,...
20282,home_run,Power Hitters,Four-Seam Aces (Elite FF Impact)
20283,single,Balanced Hitters,Balanced Positive Arsenal (Mild Impact Gains)
20284,single,Balanced Hitters,Balanced Positive Arsenal (Mild Impact Gains)
20285,walk,Balanced Hitters,Balanced Positive Arsenal (Mild Impact Gains)


In [105]:
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
clean = associationRules.loc[eventsFilter]
clean.sort_values(by='lift', ascending=False, inplace=True)
clean.sort_values(by='confidence', ascending=False, inplace=True)
display(clean)
clean.to_csv("HitterImpact2022.csv", index=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
78,(impact_archetype_Sinker Aces (Elite Sinker Im...,(events_single),0.029181,0.432297,0.014689,0.503378,1.164428,1.0,0.002074,1.143131,0.145454,0.032877,0.125209,0.268679
182,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.037561,0.432297,0.018682,0.497375,1.150542,1.0,0.002444,1.129478,0.135951,0.041407,0.114635,0.270295
170,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.112486,0.432297,0.054468,0.484224,1.120121,1.0,0.005841,1.100679,0.120831,0.111089,0.091470,0.305111
30,(Hitter_Types_Hitters With Good Contact),(events_single),0.211860,0.432297,0.102529,0.483946,1.119477,1.0,0.010942,1.100085,0.135415,0.189297,0.090980,0.360559
177,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.038892,0.432297,0.017893,0.460076,1.064260,1.0,0.001080,1.051451,0.062824,0.039474,0.048933,0.250734
80,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.044068,0.432297,0.019323,0.438479,1.014301,1.0,0.000272,1.011010,0.014749,0.042278,0.010890,0.241588
72,(impact_archetype_Balanced Positive Arsenal (M...,(events_single),0.532016,0.432297,0.231084,0.434356,1.004763,1.0,0.001095,1.003640,0.010130,0.315160,0.003627,0.484453
130,"(Hitter_Types_Balanced Hitters, impact_archety...",(events_single),0.105536,0.432297,0.045744,0.433442,1.002650,1.0,0.000121,1.002022,0.002955,0.092958,0.002018,0.269629
122,(impact_archetype_Balanced Positive Arsenal (M...,(events_single),0.285158,0.432297,0.123577,0.433362,1.002465,1.0,0.000304,1.001881,0.003440,0.208084,0.001877,0.359612
10,(Hitter_Types_Balanced Hitters),(events_single),0.538522,0.432297,0.232267,0.431304,0.997705,1.0,-0.000534,0.998255,-0.004960,0.314490,-0.001748,0.484295


In [106]:
#Third One 
#(Hitter_Types + Ball Preference + Impact Archetype)
transactions22Part3 = complete[['events','Hitter_Types','Ball Preference','impact_archetype']].copy()
display(transactions22Part3)
for col in transactions22Part3.columns:
    transactions22Part3[col] = transactions22Part3[col].astype(str)
    transactions22Part3[col] = col + "_" + transactions22Part3[col]
transactionListsPart3 = transactions22Part3.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart3 = EncodeData.fit(transactionListsPart3).transform(transactionListsPart3)
EncodedInformation = pd.DataFrame(listofItemsPart3, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)


,events,Hitter_Types,Ball Preference,impact_archetype
0,single,Balanced Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
1,single,Hitters With Good Contact,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
2,walk,Balanced Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
3,home_run,Power Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
4,single,Balanced Hitters,Selective / Moderate,Four-Seam Aces (Elite FF Impact)
...,...,...,...,...
20282,home_run,Power Hitters,Selective / Moderate,Four-Seam Aces (Elite FF Impact)
20283,single,Balanced Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
20284,single,Balanced Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)
20285,walk,Balanced Hitters,Selective / Moderate,Balanced Positive Arsenal (Mild Impact Gains)


In [107]:
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
cleansedDF = associationRules.loc[eventsFilter]
cleansedDF.sort_values(by='lift', ascending=False, inplace=True)
cleansedDF.sort_values(by='confidence', ascending=False, inplace=True)

display(cleansedDF)
cleansedDF.to_csv("HitteBallImpact2022.csv", index=False)
   

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
138,(impact_archetype_Sinker Aces (Elite Sinker Im...,(events_single),0.029181,0.432297,0.014689,0.503378,1.164428,1.0,0.002074,1.143131,0.145454,0.032877,0.125209,0.268679
824,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.025041,0.432297,0.012520,0.500000,1.156613,1.0,0.001695,1.135407,0.138885,0.028147,0.119258,0.264481
638,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.037561,0.432297,0.018682,0.497375,1.150542,1.0,0.002444,1.129478,0.135951,0.041407,0.114635,0.270295
796,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.075171,0.432297,0.037117,0.493770,1.142203,1.0,0.004621,1.121435,0.134618,0.078914,0.108285,0.289816
212,"(Hitter_Types_Hitters With Good Contact, Ball ...",(events_single),0.141963,0.432297,0.070094,0.493750,1.142156,1.0,0.008724,1.121390,0.145055,0.139030,0.108249,0.327947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,(Hitter_Types_Balanced Hitters),(events_sac_fly),0.538522,0.019668,0.011042,0.020503,1.042489,1.0,0.000450,1.000853,0.088319,0.020180,0.000852,0.290953
130,(impact_archetype_Balanced Positive Arsenal (M...,(events_sac_fly),0.532016,0.019668,0.010697,0.020106,1.022263,1.0,0.000233,1.000447,0.046535,0.019772,0.000447,0.281983
8,(Ball Preference_Aggressive Fastball),(events_field_error),0.533445,0.018288,0.010351,0.019405,1.061098,1.0,0.000596,1.001139,0.123416,0.019120,0.001138,0.292721
120,(impact_archetype_Balanced Positive Arsenal (M...,(events_field_error),0.532016,0.018288,0.010302,0.019364,1.058883,1.0,0.000573,1.001098,0.118826,0.019078,0.001097,0.291353


In [108]:
#2023 Data
transactions23 = complete2[['events','Swinging Tendencies','quality_archetype']].copy()
for col in transactions23.columns:
    transactions23[col] = transactions23[col].astype(str)
    transactions23[col] = col + "_" + transactions23[col]
transactionLists = transactions23.values.tolist()
EncodeData= TransactionEncoder()
listofItems = EncodeData.fit(transactionLists).transform(transactionLists)
EncodedInformation = pd.DataFrame(listofItems, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
firstCleaned = associationRules.loc[eventsFilter]
firstCleaned.sort_values(by='lift', ascending=False, inplace=True)
firstCleaned.sort_values(by='confidence', ascending=False, inplace=True)
firstCleaned.to_csv("SwingQuality2023.csv", index=False)


display(firstCleaned)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
146,"(quality_archetype_Sinkers Get Hit (Bad SI), S...",(events_single),0.020924,0.420644,0.010606,0.506912,1.205086,1.0,0.001805,1.174955,0.173821,0.024611,0.148904,0.266064
128,(quality_archetype_Fastball Generalists (Good ...,(events_single),0.022274,0.420644,0.011185,0.502165,1.193799,1.0,0.001816,1.163750,0.166036,0.025907,0.140709,0.264377
134,(quality_archetype_Four-Seam Aces (Elite FF Qu...,(events_single),0.033748,0.420644,0.015958,0.472857,1.124126,1.0,0.001762,1.099049,0.114277,0.036398,0.090123,0.255397
218,(quality_archetype_Fastball Generalists (Good ...,(events_single),0.028638,0.420644,0.013355,0.466330,1.108609,1.0,0.001308,1.085607,0.100857,0.030635,0.078856,0.249039
236,"(quality_archetype_Sinkers Get Hit (Bad SI), S...",(events_single),0.027818,0.420644,0.012824,0.461005,1.095951,1.0,0.001123,1.074882,0.090055,0.029438,0.069665,0.245746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,(quality_archetype_League-Average Pitch Quality),(events_home_run),0.394562,0.106499,0.042474,0.107649,1.010800,1.0,0.000454,1.001289,0.017648,0.092620,0.001287,0.253236
49,(Swinging Tendencies_Selective),(events_home_run),0.379182,0.106499,0.032977,0.086968,0.816605,1.0,-0.007406,0.978608,-0.265651,0.072843,-0.021859,0.198305
214,(quality_archetype_League-Average Pitch Qualit...,(events_home_run),0.147527,0.106499,0.011908,0.080719,0.757932,1.0,-0.003803,0.971956,-0.272542,0.049184,-0.028853,0.096267
78,(quality_archetype_League-Average Pitch Quality),(events_hit_by_pitch),0.394562,0.028879,0.011233,0.028470,0.985857,1.0,-0.000161,0.999580,-0.023146,0.027251,-0.000421,0.208726


In [109]:
#Second One (Hitter_Types + Impact Archetype)
transactions23Part2 = complete2[['events','Hitter_Types','impact_archetype']].copy()
for col in transactions23Part2.columns:
    transactions23Part2[col] = transactions23Part2[col].astype(str)
    transactions23Part2[col] = col + "_" + transactions23Part2[col]
transactionListsPart2 = transactions23Part2.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart2 = EncodeData.fit(transactionListsPart2).transform(transactionListsPart2)
EncodedInformation = pd.DataFrame(listofItemsPart2, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
clean = associationRules.loc[eventsFilter]
clean.sort_values(by='lift', ascending=False, inplace=True)
clean.sort_values(by='confidence', ascending=False, inplace=True)
clean.to_csv("HitterImpact2023.csv", index=False)


display(clean)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
172,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.099267,0.420644,0.049513,0.498786,1.185767,1.0,0.007757,1.155905,0.173929,0.105258,0.134877,0.308247
26,(Hitter_Types_Hitters With Good Contact),(events_single),0.174573,0.420644,0.084948,0.486606,1.156811,1.0,0.011515,1.128482,0.164224,0.166478,0.113854,0.344277
130,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.056793,0.420644,0.026323,0.463497,1.101876,1.0,0.002434,1.079875,0.098024,0.058352,0.073967,0.263038
179,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.027963,0.420644,0.012872,0.460345,1.094381,1.0,0.001110,1.073567,0.088722,0.029542,0.068526,0.245473
86,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.071160,0.420644,0.031482,0.442412,1.051749,1.0,0.001549,1.039039,0.052972,0.068391,0.037572,0.258627
84,(impact_archetype_Sinker Aces (Elite Sinker Im...,(events_single),0.034085,0.420644,0.014946,0.438472,1.042383,1.0,0.000608,1.031750,0.042095,0.033984,0.030773,0.237001
82,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.111416,0.420644,0.048742,0.437473,1.040007,1.0,0.001875,1.029917,0.043292,0.100848,0.029048,0.276673
136,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.035050,0.420644,0.015235,0.434663,1.033327,1.0,0.000491,1.024797,0.033424,0.034588,0.024197,0.235440
6,(Hitter_Types_Balanced Hitters),(events_single),0.495565,0.420644,0.211310,0.426403,1.013691,1.0,0.002854,1.010041,0.026776,0.299774,0.009941,0.464376
221,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.023865,0.420644,0.010076,0.422222,1.003752,1.0,0.000038,1.002731,0.003829,0.023194,0.002724,0.223088


In [110]:
#Third One 
#(Hitter_Types + Ball Preference + Impact Archetype)
transactions23Part3 = complete2[['events','Hitter_Types','Ball Preference','impact_archetype']].copy()
for col in transactions23Part3.columns:
    transactions23Part3[col] = transactions23Part3[col].astype(str)
    transactions23Part3[col] = col + "_" + transactions23Part3[col]
transactionListsPart3 = transactions23Part3.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart3 = EncodeData.fit(transactionListsPart3).transform(transactionListsPart3)
EncodedInformation = pd.DataFrame(listofItemsPart3, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
cleansedDF = associationRules.loc[eventsFilter]
cleansedDF.sort_values(by='lift', ascending=False, inplace=True)
cleansedDF.sort_values(by='confidence', ascending=False, inplace=True)
cleansedDF.to_csv("HitterBallImpact2023.csv", index=False)


display(cleansedDF)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
700,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.037798,0.420644,0.020152,0.533163,1.267493,1.0,0.004253,1.241025,0.219331,0.045980,0.194214,0.290536
198,"(Hitter_Types_Hitters With Good Contact, Ball ...",(events_single),0.067158,0.420644,0.034326,0.511127,1.215106,1.0,0.006077,1.185085,0.189771,0.075696,0.156179,0.296366
606,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.099267,0.420644,0.049513,0.498786,1.185767,1.0,0.007757,1.155905,0.173929,0.105258,0.134877,0.308247
82,(Hitter_Types_Hitters With Good Contact),(events_single),0.174573,0.420644,0.084948,0.486606,1.156811,1.0,0.011515,1.128482,0.164224,0.166478,0.113854,0.344277
840,"(Ball Preference_Selective / Moderate, Hitter_...",(events_single),0.061469,0.420644,0.029361,0.477647,1.135514,1.0,0.003504,1.109127,0.127157,0.064849,0.098390,0.273723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,(Hitter_Types_Balanced Hitters),(events_hit_by_pitch),0.495565,0.028879,0.012921,0.026073,0.902834,1.0,-0.001391,0.997119,-0.175839,0.025259,-0.002889,0.236742
132,(impact_archetype_Balanced Positive Arsenal (M...,(events_sac_fly),0.574728,0.019092,0.011089,0.019294,1.010580,1.0,0.000116,1.000206,0.024617,0.019029,0.000206,0.300051
38,(Ball Preference_Selective / Moderate),(events_sac_fly),0.756533,0.019092,0.014367,0.018991,0.994703,1.0,-0.000077,0.999897,-0.021404,0.018873,-0.000103,0.385758
122,(impact_archetype_Balanced Positive Arsenal (M...,(events_field_error),0.574728,0.016681,0.010558,0.018371,1.101301,1.0,0.000971,1.001721,0.216292,0.018177,0.001718,0.325659


In [111]:
#2024 Data
transactions24 = complete3[['events','Swinging Tendencies','quality_archetype']].copy()
for col in transactions24.columns:
    transactions24[col] = transactions24[col].astype(str)
    transactions24[col] = col + "_" + transactions24[col]
transactionLists = transactions24.values.tolist()
EncodeData= TransactionEncoder()
listofItems = EncodeData.fit(transactionLists).transform(transactionLists)
EncodedInformation = pd.DataFrame(listofItems, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
firstCleaned = associationRules.loc[eventsFilter]
firstCleaned.sort_values(by='lift', ascending=False, inplace=True)
firstCleaned.to_csv("SwingQuality2024.csv", index=False)


display(firstCleaned)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
220,(quality_archetype_Four-Seam Aces (Elite FF Qu...,(events_walk),0.039014,0.238132,0.011362,0.291223,1.222948,1.0,0.002071,1.074905,0.189705,0.042748,0.069685,0.169468
226,(quality_archetype_League-Average Pitch Qualit...,(events_walk),0.154916,0.238132,0.044773,0.289015,1.213676,1.0,0.007883,1.071567,0.208330,0.128557,0.066787,0.238516
231,(quality_archetype_Weak Fastball Group (Bad FF...,(events_walk),0.054734,0.238132,0.015460,0.282464,1.186166,1.0,0.002426,1.061784,0.166036,0.055732,0.058189,0.173694
184,(quality_archetype_League-Average Pitch Qualit...,(events_home_run),0.154916,0.103554,0.018833,0.121567,1.173953,1.0,0.002791,1.020506,0.175340,0.078588,0.020094,0.151716
37,(Swinging Tendencies_Balanced),(events_walk),0.369442,0.238132,0.102672,0.277910,1.167042,1.0,0.014696,1.055087,0.226994,0.203350,0.052211,0.354533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,(quality_archetype_Weak Fastball Group (Bad FF...,(events_single),0.054734,0.431647,0.019715,0.360190,0.834454,1.0,-0.003911,0.888314,-0.173469,0.042246,-0.125728,0.202931
166,(quality_archetype_League-Average Pitch Qualit...,(events_walk),0.141427,0.238132,0.027289,0.192957,0.810292,1.0,-0.006389,0.944023,-0.214262,0.077467,-0.059296,0.153777
9,(Swinging Tendencies_Aggressive),(events_walk),0.338833,0.238132,0.065214,0.192467,0.808234,1.0,-0.015473,0.943451,-0.264088,0.127433,-0.059939,0.233161
109,(quality_archetype_Fastball Generalists (Good ...,(events_walk),0.077925,0.238132,0.014994,0.192410,0.807997,1.0,-0.003563,0.943384,-0.204905,0.049802,-0.060013,0.127687


In [112]:
#Second One (Hitter_Types + Impact Archetype)
transactions24Part2 = complete3[['events','Hitter_Types','impact_archetype']].copy()
for col in transactions24Part2.columns:
    transactions24Part2[col] = transactions24Part2[col].astype(str)
    transactions24Part2[col] = col + "_" + transactions24Part2[col]
transactionListsPart2 = transactions24Part2.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart2 = EncodeData.fit(transactionListsPart2).transform(transactionListsPart2)
EncodedInformation = pd.DataFrame(listofItemsPart2, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)
clean = associationRules.loc[eventsFilter]
clean.sort_values(by='lift', ascending=False, inplace=True)
clean.sort_values(by='confidence', ascending=False, inplace=True)
clean.to_csv("HitterImpact2024.csv", index=False)


display(clean)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
160,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.112166,0.431647,0.056654,0.505088,1.170140,1.0,0.008238,1.148391,0.163771,0.116294,0.129217,0.318169
28,(Hitter_Types_Hitters With Good Contact),(events_single),0.184280,0.431647,0.090947,0.493525,1.143352,1.0,0.011403,1.122173,0.153703,0.173238,0.108872,0.352111
78,(impact_archetype_Sinker Aces (Elite Sinker Im...,(events_single),0.029364,0.431647,0.014475,0.492933,1.141981,1.0,0.001800,1.120863,0.128090,0.032415,0.107830,0.263233
130,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.030869,0.431647,0.014682,0.475630,1.101896,1.0,0.001358,1.083878,0.095419,0.032785,0.077387,0.254822
167,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.031855,0.431647,0.014578,0.457655,1.060252,1.0,0.000828,1.047954,0.058698,0.032474,0.045759,0.245714
80,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.064436,0.431647,0.029364,0.455717,1.055762,1.0,0.001551,1.044222,0.056454,0.062917,0.042349,0.261873
124,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.044099,0.431647,0.020026,0.454118,1.052057,1.0,0.000991,1.041164,0.051764,0.043944,0.039536,0.250256
76,(impact_archetype_Four-Seam Aces (Elite FF Imp...,(events_single),0.092192,0.431647,0.040519,0.439505,1.018204,1.0,0.000724,1.014019,0.019694,0.083834,0.013825,0.266687
8,(Hitter_Types_Balanced Hitters),(events_single),0.472944,0.431647,0.204358,0.432097,1.001043,1.0,0.000213,1.000793,0.001977,0.291843,0.000792,0.452767
72,(impact_archetype_Balanced Positive Arsenal (M...,(events_single),0.601505,0.431647,0.259767,0.431861,1.000496,1.0,0.000129,1.000377,0.001244,0.335882,0.000377,0.516832


In [113]:
#Third One 
#(Hitter_Types + Ball Preference + Impact Archetype)
transactions24Part3 = complete3[['events','Hitter_Types','Ball Preference','impact_archetype']].copy()
for col in transactions24Part3.columns:
    transactions24Part3[col] = transactions24Part3[col].astype(str)
    transactions24Part3[col] = col + "_" + transactions24Part3[col]
transactionListsPart3 = transactions24Part3.values.tolist()
EncodeData= TransactionEncoder()
listofItemsPart3 = EncodeData.fit(transactionListsPart3).transform(transactionListsPart3)
EncodedInformation = pd.DataFrame(listofItemsPart3, columns=EncodeData.columns_)
frequentItemSetUsed = apriori(EncodedInformation, min_support = 0.01, use_colnames = True)
associationRules = association_rules(frequentItemSetUsed,metric='confidence',min_threshold = 0.01)
eventsFilter = []
for i in range(len(associationRules)):
    antecedent_items = associationRules.loc[i, 'antecedents']
    consequent_items = associationRules.loc[i, 'consequents']
    if len(consequent_items) != 1:
        continue
    if any('events' in item for item in consequent_items):
        eventsFilter.append(i)

cleansedDF = associationRules.loc[eventsFilter]
cleansedDF.sort_values(by='lift', ascending=False, inplace=True)
cleansedDF.sort_values(by='confidence', ascending=False, inplace=True)
cleansedDF.to_csv("HitterBallImpact2024.csv", index=False)

display(cleansedDF)



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
180,"(Hitter_Types_Hitters With Good Contact, Ball ...",(events_single),0.023087,0.431647,0.011881,0.514607,1.192193,1.0,0.001915,1.170912,0.165019,0.026828,0.145965,0.271065
546,"(Hitter_Types_Hitters With Good Contact, impac...",(events_single),0.112166,0.431647,0.056654,0.505088,1.170140,1.0,0.008238,1.148391,0.163771,0.116294,0.129217,0.318169
788,"(Ball Preference_Selective / Moderate, Hitter_...",(events_single),0.097276,0.431647,0.048197,0.495467,1.147851,1.0,0.006208,1.126492,0.142687,0.100259,0.112288,0.303563
732,(impact_archetype_Sinker Liability Pitchers (S...,(events_single),0.024488,0.431647,0.012088,0.493644,1.143629,1.0,0.001518,1.122437,0.128743,0.027223,0.109082,0.260824
84,(Hitter_Types_Hitters With Good Contact),(events_single),0.184280,0.431647,0.090947,0.493525,1.143352,1.0,0.011403,1.122173,0.153703,0.173238,0.108872,0.352111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,(Hitter_Types_Balanced Hitters),(events_sac_fly),0.472944,0.019507,0.010065,0.021281,1.090948,1.0,0.000839,1.001813,0.158173,0.020865,0.001809,0.268619
36,(Ball Preference_Selective / Moderate),(events_sac_fly),0.841401,0.019507,0.016187,0.019238,0.986197,1.0,-0.000227,0.999725,-0.081090,0.019162,-0.000275,0.424513
126,(impact_archetype_Balanced Positive Arsenal (M...,(events_sac_fly),0.601505,0.019507,0.011569,0.019234,0.986003,1.0,-0.000164,0.999722,-0.034399,0.018984,-0.000278,0.306160
30,(Ball Preference_Selective / Moderate),(events_field_error),0.841401,0.016031,0.013904,0.016525,1.030798,1.0,0.000415,1.000502,0.188383,0.016483,0.000502,0.441919
